# CIFAR-10

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import sys

sys.path.append('../')
sys.path.append('../src/')

import src.general as general
import src.dataset_models as data
import src.metrics as metrics
import src.evaluation as eval
import src.plot as plot
import src.compression.distillation as distill
import src.compression.pruning as prune


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [11]:
dataset = data.supported_datasets["CIFAR-10"]

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [22]:
import models.resnet as rs


resnet = rs.resnet18()

# Replace the last fully connected layer with a new one
num_features = resnet.fc.in_features
resnet.fc = nn.Linear(num_features, 10)


In [23]:
# Load the fine-tuned weights
resnet.load_state_dict(torch.load('../models/resnet18.pt'))

<All keys matched successfully>

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

Evaluate before compressing

In [24]:
before_results = eval.get_results(resnet, dataset)

plot.print_results(**before_results)

Test: 100%|██████████| 157/157 [00:22<00:00,  6.86it/s]


Test loss: -5.2595
Test score: 92.6254
Could not calculate FLOPS
==================================== RESULTS ===================================
Loss: -5.259476
Score: 92.625398
Time per data point: 9.1134 ms
Model Size: 42.69 MB
Number of parameters: 11173962
Number of FLOPs: -1
Number of MACs: 140848128


# Compression

## Pruning

In [25]:
prune.magnitude_pruning_structured(resnet,dataset, sparsity=0.2, fineTune=True)

Train:  20%|██        | 157/782 [01:37<06:29,  1.60it/s]


KeyboardInterrupt: 

Structured pruning example.

In [16]:
prune.magnitude_pruning_structured(resnet,dataset, sparsity=0.8, fineTune=True)

Train: 100%|██████████| 782/782 [05:24<00:00,  2.41it/s]


Train loss: -428658.7833
Train score: 27.2498


Train: 100%|██████████| 782/782 [03:41<00:00,  3.53it/s]


Train loss: -6314159519977.1807
Train score: 11.7587


Train: 100%|██████████| 782/782 [02:01<00:00,  6.45it/s]

Train loss: -68383201205360295936.0000
Train score: 9.9365


ResNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## Distillation

In [18]:
settings = {
    # "performance_target": 99,
    "fineTune": False,
    "epochs": 3,
}

distilled_model = distill.perform_distillation(resnet, dataset, settings)

Settings: {'fineTune': False, 'epochs': 3}
============================= CREATING STUDENT MODEL ===========================
Fine-tuning: False


============================ PERFORMING DISTILLATION ===========================
Epoch: 0


Distillation Training:   1%|          | 5/782 [00:01<04:59,  2.59it/s]


KeyboardInterrupt: 

In [ ]:
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  